# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import os
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,31.59,31,0,3.37,AU,1673244592
1,1,iqaluit,63.7506,-68.5145,-30.17,67,51,12.96,CA,1673244633
2,2,neryungri,56.6709,124.6491,-19.19,93,100,1.08,RU,1673244634
3,3,acarau,-2.8856,-40.1200,25.66,80,96,5.10,BR,1673244634
4,4,chernyshevskiy,63.0128,112.4714,-40.32,87,62,1.17,RU,1673244634


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter("Lng", "Lat", frame_width=700, frame_height=500, size="Humidity", by="Country")

# Display the map
map_plot

:NdOverlay   [Country]
   :Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal = city_data_df.loc[(city_data_df["Max Temp"]>=15) & (city_data_df["Max Temp"]<=30) & (city_data_df["Humidity"]<=50) & (city_data_df["Cloudiness"]<=20)]

# Drop any rows with null values
city_clean = city_ideal.dropna()

# Display sample data
city_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,tezu,27.9167,96.1667,24.64,20,0,1.73,IN,1673244636
19,19,marzuq,14.4000,46.4667,20.74,49,3,0.49,YE,1673244639
29,29,mataura,-46.1927,168.8643,25.28,41,0,3.12,NZ,1673244643
30,30,balapur,20.6667,76.7667,24.36,17,0,1.86,IN,1673244644
33,33,pasighat,28.0667,95.3333,23.80,25,0,1.83,IN,1673244645
46,46,tura,25.5198,90.2201,24.78,31,0,1.55,IN,1673244650
110,110,kalavad,22.2167,70.3833,26.26,42,0,4.10,IN,1673244675
142,142,dembi dolo,8.5333,34.8000,19.77,40,18,1.69,ET,1673244692
214,214,narsingdi,23.9208,90.7181,21.01,29,0,2.53,BD,1673244722
226,226,chodavaram,17.8333,82.9500,26.91,25,0,2.57,IN,1673244604


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_clean[["Lat", "Lng", "City", "Country"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\jonow\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Lat,Lng,City,Country,Hotel Name
11,27.9167,96.1667,tezu,IN,
19,14.4000,46.4667,marzuq,YE,
29,-46.1927,168.8643,mataura,NZ,
30,20.6667,76.7667,balapur,IN,
33,28.0667,95.3333,pasighat,IN,
46,25.5198,90.2201,tura,IN,
110,22.2167,70.3833,kalavad,IN,
142,8.5333,34.8000,dembi dolo,ET,
214,23.9208,90.7181,narsingdi,BD,
226,17.8333,82.9500,chodavaram,IN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (788381735.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE